In [77]:
import numpy as np 
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import copy
import itertools
import os
import csv

In [78]:
# Hand detection model initialization
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    max_num_hands = 2,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.5,
)

I0000 00:00:1724308775.064007 8173610 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro


In [79]:
img_path = "/Users/Atharv/All scripts/SmartMAte/ASL Translation/model/keypoint_classifier/ASL_Dataset/asl_alphabet_train/asl_alphabet_train/"
data = []
for i in range(65,91):
    for j in range(1,3001):
        final_path = img_path + chr(i) + "/" + chr(i) + str(j) + ".jpg"
        image = cv2.imread(final_path)
        data.append(image)       

print(len(data))

W0000 00:00:1724308775.074505 8225779 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724308775.082400 8225778 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


78000


In [80]:
def calc_landmark_points(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_points = []

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_points.append([landmark_x, landmark_y])
    
    return landmark_points

In [81]:
def preprocess_keypoints(landmarks):
    temp_landmark_list = copy.deepcopy(landmarks)

    # Converting into local coordinates
    x, y = 0,0
    for index, landmark_point in enumerate(landmarks):
        if index == 0:
            x,y = landmark_point[0],landmark_point[1]
        
        temp_landmark_list[index][0] = temp_landmark_list[index][0] - x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - y

    # Converting into 1D  list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    # Finding the max value for normalization 
    maximum = max(list(map(abs, temp_landmark_list)))
    def normalization(n):
        return n/maximum
    temp_landmark_list = list(map(normalization, temp_landmark_list))
    
    return temp_landmark_list

In [82]:
def csv_logger(num, pre_processed_keypoints):
    
    csv_path = "/Users/Atharv/All scripts/SmartMAte/ASL Translation/model/keypoint_classifier/keypoints.csv"
    with open(csv_path, 'rb+') as f:
        f.seek(-1, os.SEEK_END)
        last_char = f.read(1)
        if last_char != b'\n':
            f.write(b'\n')
    with open(csv_path, 'a', newline = "") as f:
        writer = csv.writer(f)
        writer.writerow([num, *pre_processed_keypoints])
    return

In [107]:
csv_path = "/Users/Atharv/All scripts/SmartMAte/ASL Translation/model/keypoint_classifier/keypoints.csv"
pointer = 0
num = 9
while pointer < len(data):
    temp = pointer / 3000
    if temp.is_integer():
        num += 1
        print(num)
    if pointer == 0:
        print("entered")
        pointer += 1
    image = data[pointer]
    results = hands.process(image)
    if results.multi_hand_landmarks:
        # Extracting hand landmarks
        hand_landmarks = results.multi_hand_landmarks
        # Calculating landmark points
        landmark_points = calc_landmark_points(image, hand_landmarks[0])
        # Localizing landmark_points
        pre_processed_keypoints = preprocess_keypoints(landmark_points)
        # Writing it to csv file 
        csv_logger(num, pre_processed_keypoints)
    pointer += 1

10
entered


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
